In [71]:
import requests
import json
from bs4 import BeautifulSoup
from csv import writer

client_id1 = 'client_660b4d0cceb4cf9f989acef154caf4ab'
client_secret1 = 'secret_bd749d4e48d050deb1d8873b061456f2'
client_id2 = 'client_ccde1b02ebfe494290bc6156fd6e8e98'
client_secret2 = 'secret_f3fd91ad80aaced9d4f4d53fef4f7ebb'
client_id3 = 'client_aba6e5dec04c5d1a2335705504e56729'
client_secret3 = 'secret_cabc2a80fd7cd09a918f86b5672962cb'
client_id4 = 'client_b74a3bc9756e77bdcd0a2ffb2e430edb'
client_secret4 = 'secret_666cf9388a8462e398aadf7d4d105879'

client = {}
client['client_id'] = [client_id1, client_id2, client_id3, client_id4],
client['client_secret'] = [client_secret1, client_secret2, client_secret3, client_secret4]

def retrive_properties(client_id, client_secret, pageNumber, postCode="", state="VIC", pageSize=200, filename="propertiesInfo.csv"):
  scopes = ['api_listings_read']
  auth_url = 'https://auth.domain.com.au/v1/connect/token'
  url_endpoint = 'https://api.domain.com.au/v1/listings/residential/_search'

  response = requests.post(auth_url, data = {
                          'client_id':client_id,
                          'client_secret':client_secret,
                          'grant_type':'client_credentials',
                          'scope':scopes,
                          'Content-Type':'text/json'
                          })
  json_res = response.json()
  access_token=json_res['access_token']
  print("Access:", access_token)
  auth = {'Authorization':'Bearer ' + access_token}
  url = url_endpoint

  post_fields = {
        "listingType":"Rent",
        "pageSize": pageSize, 
        "pageNumber": pageNumber, # 10 per page
        "propertyTypes":"",
        "minBedrooms":1,
        "minBathrooms":1,
        # "maxPrice": 400,
        "sort": {
            "sortKey": "Default"
        },

        "locations":[
          {
            "state":state,
            "region":"",
            "area":"",
            "suburb":"",
            "postCode":postCode,
            "includeSurroundingSuburbs":False
          }
        ]
  }

  # Request content
  respond = requests.post(url,headers=auth,json=post_fields)
  content = json.loads(respond.text)
  print("Loaded data: " + str(len(content)))
  print(content)
  if "errors" in content or len(content) == 0:
    print("Limit notice")
    print(content)
    return 1
  elif content["type"] != "PropertyListing":
    print("Error accurs")
    print(content)
    return 2

  # Get request for each property ID and extract attributes
  for i in content:
      #get details
      id = i['listing']['id']
      listing_type = i['listing']['listingType']
      r = i['listing']['propertyDetails']
      street_address=r['displayableAddress']
      suburb=r['suburb']
      postcode=r['postcode']
      area=r['area']
      latitude = r['latitude']
      longitude = r['longitude']
      property_type=r['propertyType']
      bathrooms=r['bathrooms']
      bedrooms=r['bedrooms']

      if(str(i).find('carspaces'))>0:
          carspaces=i['listing']['propertyDetails']['carspaces']
      else:
          carspaces=0

      price=i['listing']['priceDetails']['displayPrice'] #[1:4]

      info = [id, listing_type, price, property_type, area, bedrooms, bathrooms, carspaces, street_address, suburb, postcode, latitude, longitude]

      with open(filename, 'a', newline='') as g:
          thewriter = writer(g)
          thewriter.writerow(info)

  return 0

# retrive_properties(client_id, client_secret, 1)

Rrtrive rental data

In [72]:

# Retrive rental data by postcode
# postcode 3000~4000
count = 0

for i in range(3000, 4000):
    print("Retriving data from postcode: " + str(i))
    client_id = client['client_id'][count]
    client_secret = client['client_secret'][count]

    for p in range(1, 6):
        num = retrive_properties(client_id, client_secret, p, str(i))
        if num == 1:
            break
        elif num == 2:
            count += 1
            retrive_properties(client_id, client_secret, p, str(i))

# postcode 8000~9000
for i in range(8000, 9000):
    print("Retriving data from postcode: " + str(i))
    client_id = client['client_id'][count]
    client_secret = client['client_secret'][count]
    for p in range(1, 6):
        num = retrive_properties(client_id, client_secret, p, str(i))
        if num == 1:
            break
        elif num == 2:
            count += 1
            retrive_properties(client_id, client_secret, p, str(i))


Retriving data from postcode: 3000
Access: 880c20e30f5848966a46c6d7d398ab6c
Loaded data: 3
{'type': 'https://developer.domain.com.au/docs/latest/conventions/access', 'title': 'Not Authorized', 'detail': 'Project [Sandbox] restricted from access to this environment [Primary].'}
Error accurs
{'type': 'https://developer.domain.com.au/docs/latest/conventions/access', 'title': 'Not Authorized', 'detail': 'Project [Sandbox] restricted from access to this environment [Primary].'}
Access: 6a4cfb47f5f179ba25f657a53d978bf6
Loaded data: 3
{'type': 'https://developer.domain.com.au/docs/latest/conventions/access', 'title': 'Not Authorized', 'detail': 'Project [Sandbox] restricted from access to this environment [Primary].'}
Error accurs
{'type': 'https://developer.domain.com.au/docs/latest/conventions/access', 'title': 'Not Authorized', 'detail': 'Project [Sandbox] restricted from access to this environment [Primary].'}
Access: 19a87e7fe9b114fe9111d1b6fc0d9705
Loaded data: 3
{'type': 'https://devel

IndexError: tuple index out of range